# RAG demo level 3

In [24]:
questions = [
    "What movies are about Abby?",
    "I have seen all Star Wars movies and would like tips for something similar I can watch next.",
    "What is the most common genre of the movies where one of key figures is called Mark?",
    "Are there any movies where Prague takes major role and present city as mysterious and ancient?",
    "When movies about drugs are concerned, is it usually rather serious or funny?",
    "What are some movies featuring a strong female lead that also involve adventure?",
    "Which Western films feature outlaws riding to their doom in the American Southwest?"
]

In [25]:
import subprocess
import os
import json

original_dir = os.getcwd()
try:
    # Jump into the terraform directory
    os.chdir('terraform')

    # Get the database connection string
    SQL_DATABASE = subprocess.run(['terraform', 'output', '-raw', 'SQL_DATABASE'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_USERNAME = subprocess.run(['terraform', 'output', '-raw', 'SQL_USERNAME'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_PASSWORD = subprocess.run(['terraform', 'output', '-raw', 'SQL_PASSWORD'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_SERVER_FQDN = subprocess.run(['terraform', 'output', '-raw', 'SQL_SERVER_FQDN'], stdout=subprocess.PIPE).stdout.decode('utf-8')

    # Get the embedding model endpoint and key
    model_configurations = subprocess.run(['terraform', 'output', '-raw', 'model_configurations'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    model_config = json.loads(model_configurations)
    embedding_model = model_config["models"]["text-embedding-3-large"]
    EMBEDDINGS_ENDPOINT = embedding_model["endpoint"]
    EMBEDDINGS_KEY = embedding_model["key"]
    gpt_4o_mini_model = model_config["models"]["gpt-4o-mini"]
    GPT_4O_MINI_ENDPOINT = gpt_4o_mini_model["endpoint"]
    GPT_4O_MINI_KEY = gpt_4o_mini_model["key"]
    gpt_4o_model = model_config["models"]["gpt-4o"]
    GPT_4O_ENDPOINT = gpt_4o_model["endpoint"]
    GPT_4O_KEY = gpt_4o_model["key"]

    print(f"Using {SQL_SERVER_FQDN} as the database server")
    print(f"Using {EMBEDDINGS_ENDPOINT} as the embedding model endpoint")
    print(f"Using {GPT_4O_MINI_ENDPOINT} as the gpt-4o-mini model endpoint")
    print(f"Using {GPT_4O_ENDPOINT} as the gpt-4o model endpoint")

finally:
    os.chdir(original_dir)

Using sql-graphrag-psbv.database.windows.net as the database server
Using https://graphrag-psbv.openai.azure.com/ as the embedding model endpoint
Using https://graphrag-psbv.openai.azure.com/ as the gpt-4o-mini model endpoint
Using https://graphrag-psbv.openai.azure.com/ as the gpt-4o model endpoint


In [59]:
from openai import AzureOpenAI 
import pandas as pd
import pyodbc

# Azure SQL Entra login option
# conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER_FQDN};DATABASE={SQL_DATABASE};UID=tokubica@microsoft.com;Authentication=ActiveDirectoryInteractive;Encrypt=yes;"

# Azure SQL password login option
conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER_FQDN};DATABASE={SQL_DATABASE};UID={SQL_USERNAME};PWD={SQL_PASSWORD};Encrypt=yes;Charset=Latin1;"

conn = pyodbc.connect(conn_str)

gpt_embedding_client = AzureOpenAI(
    azure_endpoint=EMBEDDINGS_ENDPOINT,
    api_key=EMBEDDINGS_KEY,
    api_version="2025-02-01-preview",
)

gpt_4o_client = AzureOpenAI(  
    azure_endpoint=GPT_4O_ENDPOINT,  
    api_key=GPT_4O_KEY,  
    api_version="2024-08-01-preview",
)

gpt_4o_mini_client = AzureOpenAI(
    azure_endpoint=GPT_4O_MINI_ENDPOINT,  
    api_key=GPT_4O_MINI_KEY,  
    api_version="2024-08-01-preview",
)

In [63]:
from enum import Enum
from pydantic import BaseModel, ValidationError, conlist, Field
from typing import List
import jinja2
import time

class SubQuestions(BaseModel):
    questions: List[str] = Field(description="List of specific sub-questions derived from the main user query, preferably 3 or more")

class FixedQuery(BaseModel):
    query: str = Field(description="Query to be executed in the database, pure SQL with no additional comments. All text-based queries should be hardcoded, just vector will be referenced as %q.")

class QueryStrategy(BaseModel):
    id: str = Field(
        description="Unique random identifier for this strategy"
    )
    message_to_user: str = Field(
        description="Message to be displayed to the user, explaining the purpose of this query. Include type of each query step and its relation to other steps. Wording should be about what you are doing, example: 'Searching Movie using keyword search for Star Wars followed by graph query to get related genres and semantic movie search among results."
    )
    query: str = Field(
        description="Rewrite query to better match knowledge base using semantic search. This field is used to create embedding and store as @q variable for command to use."
    )
    command: str = Field(
        description="Command to be executed in the database. This should be a SQL query that can be executed directly against the database. All text-based queries should be hardcoded, just vector will be referenced as %q."
    )


class QueryStrategies(BaseModel):
    strategies: List[QueryStrategy] = Field(
        description="List of strategies, each containing a series of steps to be executed sequentially. Each strategy is a sequence of steps, where each step depends on the results of previous steps."
    )

def get_subquestions(llm_client: AzureOpenAI, question: str) -> SubQuestions:
    with open("./prompts/subquestions.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": question},
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=SubQuestions,
        max_tokens=1000,  
        temperature=0.7,
    )
    return SubQuestions.model_validate_json(completion.choices[0].message.content)

def prepare_strategies(llm_client: AzureOpenAI, question: str) -> QueryStrategies:
    with open("./prompts/query_strategy_sql.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": question},
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=QueryStrategies,
        max_tokens=1000,  
        temperature=0.7,
    )
    return QueryStrategies.model_validate_json(completion.choices[0].message.content)

def batch_prepare_strategies(llm_client: AzureOpenAI, questions: SubQuestions) -> QueryStrategies:
    all_strategies = []
    for question in questions.questions:
        strategies = prepare_strategies(llm_client, question)
        all_strategies.extend(strategies.strategies)

    return QueryStrategies(strategies=all_strategies)

def fix_query(llm_client: AzureOpenAI, query: str, error: str) -> QueryStrategies:
    with open("./prompts/fix_query.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render(query=query, error=error)
    messages = [
        {"role": "system", "content": system_prompt}, 
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=FixedQuery,
        max_tokens=1000,  
        temperature=0.7,
    )
    # print(f"Fixing query: {query} with error: {error}")
    # print(f"LLM response: {completion.choices[0].message.content}")
    return FixedQuery.model_validate_json(completion.choices[0].message.content).query

def get_embeddings(texts):
    for attempt in range(1, 20):
        try:
            result = gpt_embedding_client.embeddings.create(
                input=texts,
                model="text-embedding-3-large",
                dimensions=1998,
            )
            break
        except Exception as e:
            if hasattr(e, "status_code") and e.status_code == 429:
                time.sleep(10)
            else:
                print(f"Embedding generation error: {e}")
                raise e
    return [json.dumps(item.embedding) for item in result.data]

def database_search(conn, query: str, command: str):
    query_embedding = get_embeddings([query])[0]
    cursor = conn.cursor()
    final_command = f"""
    DECLARE @q VECTOR(1998) = '{query_embedding}';
    {command}
    """
    
    try:
        cursor.execute(final_command)
        rows = cursor.fetchall()
    except Exception as e:
        return {"data": None, "error": e}

    columns = [column[0] for column in cursor.description]
    result = [dict(zip(columns, row)) for row in rows]
    
    return {"data": result, "error": None}

def llm_final_answer(question: str, context_jsons: list[str]) -> str:
    with open("./prompts/query.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [{"role": "system", "content": system_prompt}]
    for context_json in context_jsons:
        messages.append({"role": "user", "content": context_json})
    messages.append({"role": "user", "content": question})

    print(messages)
    
    for attempt in range(1, 20):
        try:
            completion = gpt_4o_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=1000,
                temperature=0.7,
            )
            break
        except Exception as e:
            if hasattr(e, "status_code") and e.status_code == 429:
                print(f"LLM response rate limited. Retrying in {10} seconds (Attempt {attempt}/{20})")
                time.sleep(10)
            else:
                print(f"LLM response error: {e}")
                raise e
    else:
        raise Exception("Failed to get LLM final answer after maximum retries.")
    
    return completion.choices[0].message.content

In [ ]:
from IPython.display import display, Markdown

for question in questions:
    print(f"\n\n*****************************************************\nQuestion: {question}")
    print("Generating sub-questions.")
    sub_questions = get_subquestions(gpt_4o_client, question)
    for sub_question in sub_questions.questions:
        print(f" -> Sub-question: {sub_question}")
    print("Generating query strategies.")
    strategies = batch_prepare_strategies(gpt_4o_client, sub_questions)
    context_jsons = []
    executions = strategies
    for strategy in strategies.strategies:
        print(f" -> {strategy.message_to_user}")
        results = database_search(conn, strategy.query, strategy.command)
        if not results["data"]:
            print(f"    -> Trying to fix error in query")
            for attempt in range(1,5):
                fixed_query = fix_query(gpt_4o_client, strategy.query, results["error"])
                results = database_search(conn, strategy.query, fixed_query)
                if results["data"]:
                    print(f"    -> Fixed query successful on attempt {attempt}.")
                    break
            if not results["data"]:
                print("    -> Query fix attempts failed. Moving to next strategy.")
                continue  # move on to the next strategy if no data was found
        context_json = json.dumps(results["data"])
        print(f"    -> Found {len(results["data"])} records")
        if context_json != []:
            context_jsons.append(context_json)
    llm_response = llm_final_answer(question, context_jsons)
    display(Markdown(llm_response))




*****************************************************
Question: What movies are about Abby?
Generating sub-questions.
 -> Sub-question: What are some movies with a main character named Abby?
 -> Sub-question: Which films have plots centered around someone named Abby?
 -> Sub-question: Can you list movies where the character Abby plays a significant role?
Generating query strategies.
 -> Searching for movies featuring a character named Abby using semantic search to find relevant data.
    -> Found 20 records
 -> Using keyword search to locate movies with a character named Abby, followed by filtering relevant results.
    -> Trying to fix error in query
    -> Fixed query successful on attempt 2.
    -> Found 20 records
 -> Searching Character using keyword search for 'Abby' followed by graph query to find related Movies.
    -> Trying to fix error in query
    -> Fixed query successful on attempt 1.
    -> Found 11 records
 -> Utilizing semantic search for Characters named Abby to ide

The following movies feature characters named Abby:

1. **Mean Girls 2**  
   - **Character:** Abby Hanover  
   - **Overview:** The Plastics are back in this follow-up to the original *Mean Girls*. Confident senior Jo breaks her rule of avoiding girl drama when she sees timid Abby Hanover being preyed upon by Queen Bee Mandi. Jo takes Abby's side, sparking a viciously funny girl-world-war that turns the school upside down.

2. **16 Wishes**  
   - **Character:** Abby Jensen  
   - **Overview:** Abby Jensen has been eagerly awaiting her 16th birthday, keeping a secret wish list since childhood. When disaster strikes on her big day, a mysterious box of magical birthday candles arrives, fulfilling her wishes—until one wish threatens to change everything.

3. **Nurse 3-D**  
   - **Character:** Abby Russell  
   - **Overview:** Abby Russell is a beautiful and dedicated nurse with a sinister side. She leads a secret life targeting and punishing dishonest men, revealing her darker motivations.

These movies feature Abby as a central or significant character, each with a unique storyline.



*****************************************************
Question: I have seen all Star Wars movies and would like tips for something similar I can watch next.
Generating sub-questions.
 -> Sub-question: What are some movies similar to Star Wars in terms of themes or genre?
 -> Sub-question: Can you recommend TV series that are similar to Star Wars?
 -> Sub-question: Are there any animated shows comparable to Star Wars?
 -> Sub-question: What are some sci-fi or space opera franchises I could explore next?
 -> Sub-question: Are there non-Star Wars movies or series directed by George Lucas or inspired by his work?
Generating query strategies.
 -> Performing a semantic search on the Theme table to locate themes similar to those in Star Wars, followed by a graph search to find movies associated with those themes.
    -> Found 20 records
 -> Conducting a semantic search on the Genre table to identify genres that align with Star Wars, then searching for movies in those genres.
    -> Found 20

If you've seen all the *Star Wars* movies and are looking for something similar, consider exploring films and series that feature intergalactic conflicts, heroic journeys, rich world-building, and themes of good versus evil. Here are some excellent suggestions that align with the *Star Wars* experience:

### Movies
1. **Galaxy Quest**  
   - *Overview*: A humorous yet heartfelt story of a group of washed-up actors from a sci-fi show who are mistaken by aliens for real space heroes. They are thrust into an intergalactic battle and must rise to the occasion.  
   - *Why It’s Similar*: It blends sci-fi adventure with humor and has a focus on teamwork, heroism, and redemption.

2. **Star Trek (2009)**  
   - *Overview*: This reboot of the iconic franchise revitalizes the story of James Kirk and Spock as they face a dangerous threat to the galaxy while navigating their budding partnership.  
   - *Why It’s Similar*: It features epic space battles, strong character dynamics, and an exploration of hope and unity in the face of adversity.

3. **TRON: Legacy**  
   - *Overview*: Sam Flynn searches for his missing father and is pulled into "The Grid," a stunning virtual world. Together, they must confront a malevolent force and escape.  
   - *Why It’s Similar*: The visual spectacle, the father-son dynamic, and themes of rebellion against oppressive forces echo *Star Wars* elements.

4. **The Matrix Saga**  
   - *Overview*: A hacker named Neo discovers he lives in a simulated reality controlled by machines and joins a rebellion to free humanity.  
   - *Why It’s Similar*: The "chosen one" narrative and philosophical undertones about fate, choice, and identity parallel the Jedi journey.

5. **Interstella 5555: The 5tory of the 5ecret 5tar 5ystem**  
   - *Overview*: Alien musicians are kidnapped and turned into pop stars on Earth, while a hero fights to restore their identities.  
   - *Why It’s Similar*: The space adventure, themes of individuality versus control, and heroic rescue mission resonate with *Star Wars* narratives.

### Series
1. **Stargate SG-1**  
   - *Overview*: A team uses the Stargate to travel to distant planets, encountering alien civilizations and battling threats to Earth.  
   - *Why It’s Similar*: Like *Star Wars*, it features intergalactic politics, ancient mythologies, and a team of heroes fighting for survival.

2. **Battlestar Galactica**  
   - *Overview*: Humanity's last survivors flee across space in search of Earth while being pursued by the robotic Cylons.  
   - *Why It’s Similar*: It explores themes of survival, loyalty, and the fight against oppression with a gritty, character-focused approach.

### Classics
1. **Dark Star**  
   - *Overview*: A comedic sci-fi film about a spaceship crew tasked with destroying unstable planets while dealing with existential questions.  
   - *Why It’s Similar*: Though satirical, it touches on themes of exploration and the human condition amid the vastness of space.

2. **THX 1138**  
   - *Overview*: In a dystopian future, humans live under strict control, but one technician begins to rebel against the system.  
   - *Why It’s Similar*: Directed by George Lucas, it shares his signature exploration of rebellion against oppressive regimes.

### Animated Adventures
1. **Family Guy Presents: It's a Trap!**  
   - *Overview*: A humorous retelling of *Star Wars Episode VI: Return of the Jedi* through the lens of the *Family Guy* characters.  
   - *Why It’s Similar*: It’s a playful parody for fans of *Star Wars* looking to enjoy the saga with a comedic twist.

2. **Big Hero 6**  
   - *Overview*: A young inventor and his team of friends use technology to battle a mysterious villain threatening their city.  
   - *Why It’s Similar*: It combines action, heartfelt moments, and the idea of using one's unique talents to save the day.

### Epic Adventures
1. **The Chronicles of Riddick**  
   - *Overview*: Anti-hero Riddick fights against the brutal Necromongers in a sprawling space opera.  
   - *Why It’s Similar*: The epic battles, complex worlds, and themes of survival and rebellion mirror *Star Wars*.

By exploring these options, you'll find plenty of intergalactic journeys, heroic arcs, and compelling conflicts to keep you entertained!



*****************************************************
Question: What is the most common genre of the movies where one of key figures is called Mark?
Generating sub-questions.
 -> Sub-question: What are some popular movies where one of the key characters is named Mark?
 -> Sub-question: What are the genres of these movies?
 -> Sub-question: Which genre is most common among movies featuring a key character named Mark?
Generating query strategies.
 -> Searching for popular movies featuring a character named Mark using keyword search on Character followed by graph search to identify related movies.
    -> Trying to fix error in query
    -> Fixed query successful on attempt 1.
    -> Found 20 records
 -> Using semantic search to find characters named Mark and their related movies.
    -> Trying to fix error in query
    -> Fixed query successful on attempt 1.
    -> Found 8 records
 -> Performing semantic search to identify genres related to the given movies.
    -> Found 20 records
 -> 

...

Movies about drugs often explore both serious and comedic tones, depending on the narrative focus and thematic goals. Here's a breakdown of the two approaches:

### **Serious Tone**
Many films adopt a serious tone to delve into the darker aspects of drug use, trafficking, and addiction. These movies typically aim to provoke thought, highlight social issues, or portray the devastating consequences of drug involvement. Examples include:
- **"Traffic"**: Explores the complexities and societal impact of the war on drugs through multiple perspectives.
- **"Requiem for a Dream"**: Offers a harrowing look at addiction and its destructive effects on individuals and relationships.
- **"Trainspotting"**: Depicts the chaos of heroin addiction and its impact on friendships and personal growth.
- **"The Basketball Diaries"**: Chronicles the descent into drug addiction and its life-altering consequences.
These films often evoke emotions like sadness, empathy, and reflection, emphasizing the gravity of drug-related struggles.

### **Comedic Tone**
On the other hand, some films use humor to address drug culture and its absurdities, often focusing on stoner lifestyles, misadventures, and lighthearted escapades. These movies aim to entertain rather than educate, showcasing the carefree attitudes and mishaps of characters involved with drugs. Examples include:
- **"Up in Smoke"**: Follows two stoners on a series of humorous misadventures.
- **"Half Baked"**: Centers on friends trying to bail out a friend while navigating hilarious situations.
- **"How High"**: Combines comedy with fantastical elements as two stoners use marijuana to succeed in college.
- **"Pineapple Express"**: Blends action and comedy as a stoner and his dealer are forced to run from criminals.
These films often evoke laughter, fun, and a sense of camaraderie among their characters.

### **Balance of Both**
Some movies incorporate both serious and comedic elements to provide a nuanced portrayal of drug-related themes:
- **"The Beach Bum"**: Features a lovable rogue navigating life with a humorous yet chaotic approach to drug use.
- **"We're the Millers"**: Uses comedy to explore drug trafficking through a fake family setup.
- **"The 51st State"**: Combines humor and action in the world of drug deals gone wrong.

### **Conclusion**
The tone of drug-related movies largely depends on the filmmaker’s intent. Serious films often aim to critique societal issues or portray the consequences of drug involvement, while comedic ones focus on entertainment and absurdity. Both approaches can resonate with audiences, offering either emotional depth or lighthearted escapism.



*****************************************************
Question: What are some movies featuring a strong female lead that also involve adventure?
Generating sub-questions.
 -> Sub-question: Which movies feature a strong female lead character in an adventure story?
 -> Sub-question: Can you list films that highlight strong female protagonists in adventurous roles?
 -> Sub-question: What are examples of adventure movies with prominent female leads?
Generating query strategies.
 -> Performing semantic search to find movies with strong female leads in adventure stories.
    -> Found 20 records
 -> Searching for movies using graph search to identify those associated with adventure themes and female characters.
    -> Trying to fix error in query
    -> Query fix attempts failed. Moving to next strategy.
 -> Searching for movies with themes and genres related to strong female protagonists and adventure using semantic search.
    -> Found 20 records
 -> Using keyword search to directly find 

Here are some movies featuring strong female leads that also incorporate themes of adventure:

1. **Tomb Raider** (2018)  
Lara Croft, a fiercely independent adventurer, pushes herself beyond her limits as she embarks on a perilous journey to the island where her father disappeared. This film blends action-packed sequences with Lara's courageous and resourceful character.

2. **Romancing the Stone**  
Joan Wilder, a romance novelist, finds herself on an unexpected adventure in South America while trying to rescue her abducted sister. Her journey is filled with danger, humor, and self-discovery as she navigates the jungle.

3. **The Extraordinary Adventures of Adèle Blanc-Sec**  
Set in the early 20th century, Adèle Blanc-Sec, a popular novelist, deals with suitors, monsters, and cops, showcasing her adventurous and resourceful spirit amidst fantastical distractions.

4. **The Long Kiss Goodnight**  
Samantha Caine, a suburban mom with a mysterious past, embarks on a thrilling journey of rediscovery after learning she's a lethal secret agent. Her transformation and determination drive this action-adventure.

5. **Thelma & Louise**  
While primarily a road movie, this film features two women on a journey of self-discovery and survival after a tragic encounter. Their cross-country adventure highlights themes of empowerment and friendship.

6. **Cutthroat Island**  
Morgan Adams, a daring pirate, leads an adventurous quest to recover a treasure map while battling her murderous uncle and British forces. Her leadership and fighting skills are central to the story.

7. **Wild**  
Cheryl Strayed sets out on a transformative adventure, hiking 1,000 miles on the Pacific Crest Trail to overcome her tragic past. Her journey is both physically and emotionally challenging.

8. **Raya and the Last Dragon**  
Raya, a lone warrior, embarks on an epic quest to find the last dragon and restore harmony to Kumandra. Her courage and determination drive this fantastical adventure.

9. **Princess Mononoke**  
San, a young human woman fiercely dedicated to protecting the forest, plays a central role in this epic tale of adventure, environmental themes, and conflict resolution.

10. **Howl's Moving Castle**  
Sophie, a young woman cursed into an old body, joins the enigmatic wizard Howl on a magical adventure to break her spell. Her resilience and determination are key to the plot.

These films showcase strong female protagonists leading thrilling adventures across various genres, from fantasy and action to drama and exploration.



*****************************************************
Question: Which Western films feature outlaws riding to their doom in the American Southwest?
Generating sub-questions.
 -> Sub-question: What are examples of Western films set in the American Southwest featuring outlaws?
 -> Sub-question: Which Western films depict outlaws meeting their doom?
 -> Sub-question: Are there notable Western films that portray outlaws riding towards their fate in the American Southwest?
Generating query strategies.
 -> Using semantic search to find Western films set in the American Southwest featuring outlaws.
    -> Found 20 records
 -> Using keyword search to identify Western films with relevant settings and themes.
    -> Trying to fix error in query
    -> Query fix attempts failed. Moving to next strategy.
 -> Searching Genre using keyword search for 'Western', then using semantic search on Movie for the query.
    -> Found 5 records
 -> Using semantic search on Movie to find films matching the qu

Several Western films from the provided context feature outlaws riding to their doom in the American Southwest, often capturing themes of moral ambiguity, redemption, and the inevitability of fate. Key examples include:

### 1. **The Wild Bunch**
   - **Overview**: This film focuses on aging outlaw Pike Bishop and his gang as they attempt one final robbery, only to discover it was a setup partly orchestrated by Pike's former partner, Deke Thornton. Fleeing to Mexican territory, the gang faces relentless pursuit, culminating in fierce gunfights and significant casualties. The Southwest setting emphasizes the gang's ultimate decline and doom, highlighting their struggle against the encroaching end of their era.

### 2. **The Outlaw Josey Wales**
   - **Overview**: Josey Wales, a lone outlaw avenging his family's murder, becomes pursued by soldiers as he travels through the Southwest. While Wales prefers solitude, he ends up protecting a group of vulnerable outcasts who are drawn to him. The film showcases the harsh landscapes of the Southwest and Wales’ journey as he confronts his enemies, ultimately riding toward a path of confrontation and redemption.

### 3. **Hang 'em High**
   - **Overview**: Marshall Jed Cooper survives a hanging and vows revenge on the lynch mob that tried to kill him. The film captures his pursuit of justice in the Southwest, blending themes of vengeance and survival as Jed confronts the men who condemned him. His journey is one of doom for his adversaries, but also of his own moral reckoning.

### 4. **Butch Cassidy and the Sundance Kid**
   - **Overview**: Butch Cassidy and Sundance Kid lead the outlaw Hole in the Wall Gang, but as civilization catches up with them, they are pursued relentlessly by a posse. The duo ultimately rides into South America to escape their doom. The film explores their fateful journey, symbolized by the endless pursuit and the inevitability of their demise amidst the desolate Southwest landscapes.

### 5. **My Name Is Nobody**
   - **Overview**: Jack Beauregard, a legendary gunslinger, wishes to retire peacefully but is coerced into one final showdown with a massive gang known as The Wild Bunch. Set in the Southwest, the film portrays the fading glory of the Old West and Jack’s ride toward his final act of defiance, symbolizing his doom yet ensuring his place in history.

### 6. **The Shootist**
   - **Overview**: John Bernard Brooks, a terminally ill gunfighter, returns to the Southwest seeking peace but is drawn into one last battle. The film explores themes of mortality and facing one’s inevitable doom against the backdrop of the rugged Southwest.

### Themes and Emotional Resonance:
These films explore the Southwest as an unforgiving yet iconic setting for tales of outlaws who face their fates. The journey toward doom often involves moral dilemmas, redemption, and an acknowledgment of the passing of an era, making the Southwest a poignant backdrop for these narratives.

In [62]:
context_jsons

['{"data": [{"Id": 981, "Name": "The Philadelphia Story"}, {"Id": 520172, "Name": "Happiest Season"}, {"Id": 520172, "Name": "Happiest Season"}, {"Id": 212, "Name": "Arsenic and Old Lace"}, {"Id": 16642, "Name": "Days of Heaven"}, {"Id": 620, "Name": "Ghostbusters"}, {"Id": 620, "Name": "Ghostbusters"}, {"Id": 11368, "Name": "Blood Simple"}, {"Id": 11219, "Name": "The Trouble with Harry"}, {"Id": 384677, "Name": "Set It Up"}, {"Id": 41402, "Name": "Let Me In"}, {"Id": 37233, "Name": "The Firm"}, {"Id": 555850, "Name": "The Holiday Calendar"}, {"Id": 9919, "Name": "How to Lose a Guy in 10 Days"}, {"Id": 20943, "Name": "The Ugly Truth"}, {"Id": 13250, "Name": "Butterfly on a Wheel"}, {"Id": 6687, "Name": "TransSiberian"}, {"Id": 18585, "Name": "The Ramen Girl"}, {"Id": 39414, "Name": "Paper Man"}, {"Id": 40205, "Name": "16 Wishes"}], "error": null}',
 '{"data": [{"MovieName": "Blood Simple"}, {"MovieName": "Blade: Trinity"}, {"MovieName": "Baby Driver"}, {"MovieName": "Arsenic and Old La